In [1]:
spark

In [2]:
#import statements
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import *
from datetime import datetime
from pyspark.sql.types import IntegerType, StructType, StructField, StringType

In [3]:
spark = SparkSession.builder.appName('Final_project').getOrCreate()
sc = spark.sparkContext

In [4]:
#Datapath
path_comments = "/user/rroongseang/bigdata/comments/"
path_users = "/user/rroongseang/bigdata/users/"

In [5]:
#Read data
df = spark.read.json(path_comments+"politics_json*")
users = spark.read.csv(path_users+"RA.2019-09.csv", inferSchema=True, header=True)
botusers = sqlContext.read.csv(path_users+'bot_userdata.csv',header=True)

In [6]:
#Convert column types to integers and timestamps

df = df.withColumn("ups", df["ups"].cast(IntegerType())) 
df = df.withColumn("downs", df["downs"].cast(IntegerType()))
df = df.withColumn("gilded", df["gilded"].cast(IntegerType()))
df = df.withColumn("score", df["score"].cast(IntegerType()))
df = df.withColumn("retrieved_on", df["retrieved_on"].cast(IntegerType()))
df = df.withColumn("controversiality", df["controversiality"].cast(IntegerType()))
df = df.withColumn('created_utc',df["created_utc"].cast(IntegerType()))
df = df.withColumn('retrieved_on',df["retrieved_on"].cast(IntegerType()))


df = df.withColumn('created_utc',to_timestamp(df["created_utc"]))
df = df.withColumn('retrieved_on',to_timestamp(df["retrieved_on"]))

In [7]:
df.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: integer (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: integer (nullable = true)
 |-- gilded: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: integer (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: integer (nullable = true)



In [8]:
#Rename a couple user columns to not duplicate with comments dataframe
users = users.selectExpr('id',"name as username", "created_utc as acct_creation",
                         "updated_on","comment_karma","link_karma")

botusers = botusers.selectExpr('username',"post_karma as link_karma", "comment_karma",
                                 "cake_day as acct_creation","is_bot")

In [9]:
import pyspark.sql.functions as sf
#Convert account creation and updated on to timestampps
users = users.withColumn('acct_creation',to_timestamp(users["acct_creation"]))
users = users.withColumn('updated_on',to_timestamp(users["updated_on"]))

#Add column is_bot
users = users.withColumn('is_bot',sf.lit('False'))

In [10]:
#Convert account creation and updated on to timestamps
botusers = botusers.withColumn('link_karma',botusers["link_karma"].cast(IntegerType()))
botusers = botusers.withColumn('comment_karma',botusers["comment_karma"].cast(IntegerType()))
botusers = botusers.withColumn('acct_creation',botusers["acct_creation"].cast(IntegerType()))
botusers = botusers.withColumn('acct_creation',to_timestamp(botusers["acct_creation"]))

In [11]:
users = users.join(botusers, users.username==botusers.username, "leftanti")

In [12]:
all_users = users.select('username','acct_creation','comment_karma','link_karma','is_bot')\
            .union(botusers.select('username','acct_creation','comment_karma','link_karma','is_bot'))

In [13]:
all_users.printSchema()

root
 |-- username: string (nullable = true)
 |-- acct_creation: timestamp (nullable = true)
 |-- comment_karma: integer (nullable = true)
 |-- link_karma: integer (nullable = true)
 |-- is_bot: string (nullable = true)



In [14]:
#Checking to see if one of the Russian banned accts is in the users list
all_users.filter(all_users.username == 'MiraranaMogra').show()

+-------------+-------------------+-------------+----------+------+
|     username|      acct_creation|comment_karma|link_karma|is_bot|
+-------------+-------------------+-------------+----------+------+
|MiraranaMogra|2015-05-14 09:58:34|          103|      4902|  True|
+-------------+-------------------+-------------+----------+------+



In [16]:
#Checking to see if one of the Russian comments show up in the comments.
banned = pd.read_csv('bot_accounts2019.csv')['bot_acct'].tolist()

df.filter(df.author.isin(banned)).select('author','body').show(truncate = False)

#THEY DO!! Nice!

+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|author      |body                                                                                                                                                                                                                                                                                                |
+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|BlackToLive |By submitting to an independent, non-profit community, the aut

In [17]:
#Combining data frames
df_combined = df.join(all_users, df.author == users.username, 'left')

In [34]:
#show sample combined dataframe
df_combined.select(df.author,df.body,df.ups,df.downs,df.created_utc,
                   all_users.acct_creation,all_users.username,all_users.is_bot)\
                    .filter(all_users.username.isNotNull()).show(5)

+------------+--------------------+---+-----+-------------------+-------------------+------------+------+
|      author|                body|ups|downs|        created_utc|      acct_creation|    username|is_bot|
+------------+--------------------+---+-----+-------------------+-------------------+------------+------+
| BlackToLive|By submitting to ...|  0| null|2016-07-05 12:58:29|2016-07-04 20:13:18| BlackToLive|  True|
|Maxwel_Terry|  sure it isnt new..| -7| null|2016-04-23 09:40:28|2016-04-19 08:46:36|Maxwel_Terry|  True|
| Peter_Hurst|There are many se...|  4| null|2016-04-05 08:53:23|2016-03-30 08:02:39| Peter_Hurst|  True|
|Maxwel_Terry|Our government is...|  1| null|2016-07-28 09:11:19|2016-04-19 08:46:36|Maxwel_Terry|  True|
|  Maineylops|This isn't surpri...|  4| null|2016-04-19 20:41:05|2016-03-18 05:16:23|  Maineylops|  True|
+------------+--------------------+---+-----+-------------------+-------------------+------------+------+
only showing top 5 rows



In [20]:
#Number of usernames matched 
df_combined.filter(users.username.isNotNull()).count()

5158836

In [21]:
#number of comments in original df
df.count()

6420881

In [22]:
#number of comments in combined df
df_combined.count()

6420881